In [1]:
import pickle
from datetime import timedelta, datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import sparse
from sklearn.linear_model import LogisticRegression
import scipy.stats as sts

# Пункт 1

In [2]:
with open('results.pkl', 'rb') as f:
    results = pickle.load(f)
    
with open('players.pkl', 'rb') as f:
    players = pickle.load(f)

with open('tournaments.pkl', 'rb') as f:
    tournaments = pickle.load(f)

In [4]:
players[13577]

{'id': 13577,
 'name': 'Владислав',
 'patronymic': 'Леонидович',
 'surname': 'Карнацевич'}

In [33]:
def find_X(mask):
    if mask is None:
        return True
    return 'X' in mask or '?' in mask

tournaments_train = []
tournaments_test = []
for key in tournaments.keys():
    if not results[key]:
        continue
    else:
        d = {}
        d['teams'] = []
        for team in results[key]:
            #print(team)
            #if team.get('mask', 1e10) != 1e10 and team['mask'] is not None:
            if not find_X(team.get('mask')):
                d['teams'].append({'team_id': team['team']['id'],
                                   'mask': team['mask'],
                                   'mask_sum' : len([i for i in team['mask'] if i == '1']),
                                   'teamMembers': [member['player']['id'] for member in team['teamMembers']]}
                                 )
        if d['teams'] != []:
            if tournaments[key]['dateStart'][:4] == '2019':
                d['id'] = key
                tournaments_train.append(d)
            if tournaments[key]['dateStart'][:4] == '2020':
                d['id'] = key
                tournaments_test.append(d)
print('train:', len(tournaments_train))
print('test:', len(tournaments_test))

train: 616
test: 160


In [34]:
# список всех участников на тренировочной выборке и список всех команд
members_list = []
teams_list = []
questions = 0
for tournament in tournaments_train:
    questions += len(tournament['teams'][0]['mask'])
    for team in tournament['teams']:
        teams_list.append(team['team_id'])
        members_list.extend(team['teamMembers'])
members_list = list(set(members_list))
teams_list = list(set(teams_list))
print(len(members_list))
print(len(teams_list))
print(questions)

57288
11343
30509


In [38]:
# посчитаем общее число строк которое понадобится в датасете
r = 0
for tournament in tournaments_train:
    #r += len(tournament['teams'][0]['mask']) * sum(len(t['teamMembers']) for t in tournament['teams'])
    len_q = len([i for i in tournament['teams'][0]['mask'] if i != 'X' and i != '?'])
    m_list = []
    for team in tournament['teams']:
        m_list.extend(team['teamMembers'])
    len_m = len(set(m_list))
    r += len_q * len_m
print(r)

20954324


In [39]:
member_2_index = {k : v for v, k in enumerate(members_list)}

In [40]:
len(member_2_index)

57288

In [41]:
def make_x(tournaments_data, r, member_2_index, questions):
    X_data = sparse.lil_matrix((r, questions + len(member_2_index)), 
                          dtype=int)
    target = []
    all_questions = 0
    i = 0
    members_size = len(member_2_index)
    for tournament in tournaments_data:
        len_mask = len(tournament['teams'][0]['mask'])
        for team in tournament['teams']:
            if len_mask != len(team['mask']):
                continue 
            for member in team['teamMembers']:
                for j in range(len(team['mask'])):
                    try:
                        X_data[i, members_size + all_questions + j] = int(team['mask'][j])
                    except ValueError:
                        continue
                    X_data[i, member_2_index[member]] = 1
                    #X_data[i, questions + members_list.index(member)] = 1
                    i += 1
                    target.append(int(team['mask'][j]))
        all_questions += len_mask
    return (X_data[:i, :], np.array(target))
        

In [42]:
X_train, y_train = make_x(tournaments_train, r, member_2_index, questions)

In [43]:
X_train.shape

(14771633, 87797)

In [44]:
 y_train.shape

(14771633,)

# Пункт № 2
Make BASELINE

In [45]:
clf = LogisticRegression(n_jobs=-1)
clf.fit(X_train, y_train)

LogisticRegression(n_jobs=-1)

In [46]:
clf.coef_[0][:].shape

(87797,)

In [47]:
rating_2019 = list(zip(members_list, clf.coef_[0][:len(members_list)]))

In [48]:
rating_2019.sort(key=lambda x: -x[1])

In [49]:
rating_2019[:4]

[(20640, 4.563756054996338),
 (147450, 4.440872500040505),
 (27403, 4.351978740594711),
 (223, 3.728471178284705)]

In [50]:
rating_2019 = [(players[line[0]], line[1]) for line in rating_2019]

##### Выведим топ 30

In [55]:
for line in rating_2019[:30]:
    print(line[0]['id'], line[0]['surname'], line[0]['name'], line[1])

20640 Мельникова Ольга 4.563756054996338
147450 Салихов Максим 4.440872500040505
27403 Руссо Максим 4.351978740594711
223 Аввакумов Илья 3.728471178284705
4270 Брутер Александра 3.6028584006607263
12785 Ишин Вадим 3.577376668403601
15723 Корнюков Александр 3.4768972244725753
39426 Андронов Антон 3.365435254198585
16325 Крапивин Алексей 3.3628704534007876
195091 Остросаблина Ольга 3.3108937792427624
213002 Досалина Ханна 3.291812489687915
49574 Хачатрян Самвел 3.25693202099296
193994 Крашенинникова Алина 3.17489855423336
35788 Швед Александр 3.157740622816865
217803 Тужушев Дамир 3.1211533246621204
217804 Аубакиров Алибек 3.1211533246621204
148558 Котов Виталий 3.115333477591819
46410 Гоголь Данил 3.115333477591819
46411 Рожковой Иван 3.115333477591819
55040 Гусаров Владимир 3.109787234769964
26383 Пхакадзе Парнаваз 3.0911914972172942
4637 Буторин Антон 3.0861621653425995
22935 Новиков Илья 3.052057434639512
19703 Мамедов Джавид 3.0476157647866313
20953 Мирзалиев Руслан 3.04761576478663

In [56]:
for line in rating_2019:
    if line[0]['name'] == "Сергей" and line[0]['surname'] == 'Николенко':
        print(line)

({'id': 22799, 'name': 'Сергей', 'patronymic': 'Игоревич', 'surname': 'Николенко'}, 1.0825638309487577)


# Пункт № 3

In [57]:
t = []
for tournament in tournaments_test:
    tt = []
    if len(tournament['teams']) > 1:
        for team in tournament['teams']:
            temp_m = [m for m in team['teamMembers'] if m in members_list]
            if len(temp_m) > 0:
                temp_t = team.copy()
                temp_t['teamMembers'] = temp_m
                tt.append(temp_t)
        t.append({"teams": tt})

In [58]:
rating_pred_2020 = []
#for tournament in tournaments_test:
for tournament in t:
    tournament_pred = []
    for team in tournament['teams']:
        #print(len(tournament['teams']))
        team_size = len(team['teamMembers'])
        
        if team_size > 0:
            X_team = sparse.lil_matrix((team_size, questions + len(member_2_index)),
                                           dtype=int)
            for i in range(team_size):
                try:
                    X_team[i, member_2_index[team['teamMembers'][i]]] = 1
                except KeyError:
                    continue
            pred = clf.predict_proba(X_team)[:, 0]
            pred = 1 - pred.prod()
            tournament_pred.append(pred)
        else:
            tournament_pred.append(0)
    rating_pred_2020.append(tournament_pred)

In [59]:
true_rating_2020 = []
#for tournament in tournaments_test:
for tournament in t:
    tour = []
    for team in tournament['teams']:
        tour.append(team['mask_sum'])
    true_rating_2020.append(tour)

### Посчитаем корреляцию

In [64]:
spearman_corr = []
kendall_corr = []
for i in range(len(rating_pred_2020)):
    spearman = sts.spearmanr(true_rating_2020[i], rating_pred_2020[i]).correlation
    kendall = sts.kendalltau(true_rating_2020[i], rating_pred_2020[i]).correlation
    if kendall is np.nan:
        kendall_corr.append(0)
    else:
        kendall_corr.append(kendall)
    if spearman is np.nan:
        spearman_corr.append(0)
    else:
        spearman_corr.append(spearman)
print('spearman_corr:' , np.mean(spearman_corr))
print('kendall_corr:' , np.mean(kendall_corr))

spearman_corr: 0.45633124862875896
kendall_corr: 0.33280092371858494
